In [2]:
from snowflake.snowpark import Session
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])

    session = Session.builder.configs(connection_params).create()
    return session

In [3]:
session = get_session()

In [5]:
    #add all imports
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.metrics import recall_score, f1_score, roc_auc_score, confusion_matrix,classification_report
    from sklearn.linear_model import SGDClassifier
    from sklearn.pipeline import Pipeline
    
    df_final = session.sql("SELECT * from {table}".format(table="predictive_maintenance_final")).to_pandas()
    
    # prepare data for training
    X = df_final.drop(['FAILURE'], axis = 1)
    y = df_final['FAILURE']
    X_train, X_test, Y_train, Y_test = train_test_split(X,y,random_state = 0, test_size=0.25, shuffle = True)
    X_train.reset_index(inplace = True, drop = True)
    Y_train.reset_index(inplace = True, drop = True)

    X_test.reset_index(inplace = True, drop = True)
    Y_test.reset_index(inplace = True, drop = True)
    
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, random_state = 0, test_size=0.25)
    
    pipeline = Pipeline([
        ('OHE', OneHotEncoder(handle_unknown = 'ignore',sparse=False)),
#         ('SCALER', StandardScaler()),
        ('SGD', SGDClassifier(random_state=0))
     ])
    
    pipeline.fit(x_train,y_train)
    y_pred = pipeline.predict(x_val) 
    cf_matrix = confusion_matrix(y_val, y_pred)
    cls_report = classification_report(y_val, y_pred, output_dict=True)
    print(cls_report)

/opt/conda/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


{'0': {'precision': 0.9991432120978452, 'recall': 1.0, 'f1-score': 0.9995714224488921, 'support': 23323.0}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 20.0}, 'accuracy': 0.9991432120978452, 'macro avg': {'precision': 0.4995716060489226, 'recall': 0.5, 'f1-score': 0.49978571122444604, 'support': 23343.0}, 'weighted avg': {'precision': 0.9982871582811996, 'recall': 0.9991432120978452, 'f1-score': 0.9987150017467982, 'support': 23343.0}}


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
x_val["LABEL"] = y_val

In [12]:
x_val["PREDICTION"] = y_pred

In [16]:
ref = x_val[:5000]

In [20]:
curr = x_val[5001:10001]

In [21]:
curr.shape

(5000, 17)

In [22]:
import pandas as pd
# data_final = pd.read_csv('/data/predictive_maintenance_dataset/pred_mnt_prepared.csv')

md = session.createDataFrame(ref.to_numpy().tolist(),
                                   schema=ref.columns.tolist())
md.write.mode("append").save_as_table("drift_ref_predictive_maintenance")

In [23]:
import pandas as pd
# data_final = pd.read_csv('/data/predictive_maintenance_dataset/pred_mnt_prepared.csv')

md = session.createDataFrame(curr.to_numpy().tolist(),
                                   schema=curr.columns.tolist())
md.write.mode("append").save_as_table("drift_cur_predictive_maintenance")